# Fonction Python et symbolique

## Résumé

Dans cette page, nous présentons comment définir des fonctions Python et symboliques.

[Pour voir ce Jupyter Notebook, nous conseillons NBViewer.](https://nbviewer.org/github/mbaudin47/otsupgalilee-eleve/blob/master/1-Intro-OT/Fonctions.ipynb)

## Références

- User Manual, Functions : http://openturns.github.io/openturns/master/user_manual/functions.html
- Examples, Functional modeling : http://openturns.github.io/openturns/master/examples/functional_modeling/functional_modeling.html

- http://openturns.github.io/openturns/master/user_manual/_generated/openturns.MemoizeFunction.html
- Sur ExprTk : OpenTURNS Users’ Day #11, Friday, the 15 th, June 2018, Denis Barbier, http://trac.openturns.org/blog/OpenTURNS_Users_Day_11

## Exemple

Pour chaque fonction, nous illustrons la connexion avec l’exemple suivant :

- 3 entrées, de loi normale standard, indépendantes,
- 2 sorties,
- formule symbolique.

La formule symbolique est donnée par :
$$
\begin{aligned}
Y_1 & = X_1 + X_2 + X_3, \\
Y_2 & = X_1 - X_2 X_3.
\end{aligned}
$$

Les résultats exacts sont les suivants.

| Variable | Espérance | Ecart-type |
|-|-|-|
| $Y_1$ | 0 | 1.732 |
| $Y_2$ | 0 | 1.415 |

**Table 1.** Espérance et écart-type des sorties du modèle physique.


In [1]:
import openturns as ot
import math
import numpy as np

In [2]:
ot.__version__

'1.26'

On enregistre ici la valeur de référence du vecteur moyen et du vecteur des écart-types.

In [3]:
trueMean = ot.Point([0.0, 0.0])
trueSD = ot.Point([1.732, 1.415])

In [4]:
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)
X3 = ot.Normal(0.0, 1.0)
inputDistribution = ot.JointDistribution((X1, X2, X3))
inputRandomVector = ot.RandomVector(inputDistribution)

Une alternative consiste à créer directement la loi normale multivariée.

In [5]:
inputDistribution = ot.Normal(3) # Crée directement un vecteur de 3 lois normales standard indépendantes

## PythonFunction : constructeur

La classe `PythonFunction` permet de créer une fonction OpenTURNS en utilisant une fonction Python créée avec l'opérateur `def`. 

Le constructeur de la classe `PythonFunction` est :

```python
PythonFunction(nbInputs, nbOutputs, pythonFunc)
```

où :

- `nbInputs` : nombre de variables d’entrée,
- `nbOutputs` : nombre de variables de sortie,
- `pythonFunc` : une fonction Python.

Le simulateur `simulator` a la séquence d'appel `y=simulator(x)` où :

- `x` : l’entrée du simulateur, un vecteur de dimension `nbInputs`,
- `y` : la sortie du simulateur, un vecteur de dimension `nbOutputs`.

## Exemple d'utilisation de la PythonFunction

Dans l'exemple suivant, on estime la moyenne par Monte-Carlo simple sur la base de 10 000 expériences.

In [6]:
def simulator(x):
    y1 = x[0] + x[1] + x[2]
    y2 = x[0] - x[1] * x[2]
    y = [y1, y2]
    return y

In [7]:
function = ot.PythonFunction(3, 2, simulator)
outputVect = ot.CompositeRandomVector(function, inputRandomVector)
sampleSize = 10000
outputSample = outputVect.getSample(sampleSize)
empiricalMean = outputSample.computeMean()
print(f"Sample mean = {empiricalMean}")
print(f"True mean   = {trueMean}")
empiricalSd = outputSample.computeStandardDeviation()
print(f"Sample SD = {empiricalSd}")
print(f"True SD   = {trueSD}")

Sample mean = [-0.0166778,-0.0123527]
True mean   = [0,0]
Sample SD = [1.73359,1.39888]
True SD   = [1.732,1.415]


La fonction `simulator` traite les données de manière scalaire. Bien que pédagogique, cette approche est extrêmement coûteuse lors d'un échantillonnage de type Monte-Carlo. En l'état, OpenTURNS doit effectuer une boucle Python pour appeler la fonction 10 000 fois. Si l'évaluation unitaire de la fonction est coûteuse, la pénalité n'est pas très grande. Sinon, une alternative consiste à vectoriser le calcul : cette approche est présentée dans la suite.

## Quel type pour x, pour y ?

| Type | Entrée $\boldsymbol{x}$ | Sortie $\boldsymbol{y}$ |
|--|--|--|
|Point (OpenTURNS) | ✓ | ✓ |
|list (Python) | - | ✓ |
|tuple (Python) | - | ✓ |
|array (NumPy) | - | ✓ |

## PythonFunction : objectifs, avantages, inconvénients

Les objectifs de la classe `PythonFunction` sont :

- Simplicité de mise en œuvre.
- Fonction directement en Python : possibilité d’utiliser tous les modules Python pour réaliser le calcul, ou la connexion.

**Avantages :**

- Utile si le simulateur est disponible en Python.
- Possibilité de vectorisation avec l’option `func_sample`.
- Peut être parallélisé sur plusieurs processeurs avec l'option `n_cpus` (voir l'exercice 8).

**Inconvénients :**

- Cette méthode ne permet pas le calcul analytique des dérivées.

## PythonFunction vectorisée : objectifs, avantages, inconvénients

La classe `PythonFunction` possède une option `func_sample` :

- Objectif : améliorer la performance en vectorisant les opérations.
- Principe : évaluer toutes les sorties en fonction de toutes les entrées en un seul appel à la fonction, sans boucle `for`.
- Implémentation : l’entrée et la sortie sont des tableaux (et non plus des vecteurs).

**Avantages :**

- Amélioration de la performance.

**Inconvénients :**

- Nécessite de vectoriser le calcul.

## Prototype

```python
def simulator (x):
    [...]
    return y

function = PythonFunction(nbInputs, nbOutputs, func_sample=simulator)
```

où :

- `x` : l’entrée du simulateur, un `Sample` de taille `nbExperiments` (`getSize()`), de dimension `nbInputs` (`getDimension()`),
- `y` : la sortie du simulateur
  - un `np.array` : `nbExperiments` lignes et `nbOutputs` colonnes,
  - un `Sample` : taille `nbExperiments` et dimension `nbOutputs`.

## PythonFunction vectorisée : exemple avec Numpy


In [8]:
def vectorizedSimulator(x):
    # Conversion Sample > Array Numpy
    xArray = np.array(x)
    x1 = xArray[:, 0]  # Extraction de la colonne 0
    x2 = xArray[:, 1]
    x3 = xArray[:, 2]
    y1 = x1 + x2 + x3
    y2 = x1 - x2 * x3
    # Empilement de deux colonnes
    y = np.column_stack((y1, y2))
    return y


vectorizedFunction = ot.PythonFunction(3, 2, func_sample=vectorizedSimulator)

In [9]:
outputVect = ot.CompositeRandomVector(vectorizedFunction, inputRandomVector)
sampleSize = 10000
outputSample = outputVect.getSample(sampleSize)
empiricalMean = outputSample.computeMean()
print(f"Sample mean = {empiricalMean}")
print(f"True mean   = {trueMean}")
empiricalSd = outputSample.computeStandardDeviation()
print(f"Sample SD = {empiricalSd}")
print(f"True SD   = {trueSD}")

Sample mean = [-0.00077008,-0.0165189]
True mean   = [0,0]
Sample SD = [1.74231,1.40859]
True SD   = [1.732,1.415]


## MemoizeFunction pour gérer l'historique

La classe `MemoizeFunction` définit un mécanisme d’historique des appels au modèle physique $g$.

| Méthodes | Fonction |
|-|-|
| `enableHistory()` | active l’historique (défaut : activé) |
| `disableHistory()` | désactive l’historique |
| `isHistoryEnabled()` | vrai si l’historique est actif |
| `clearHistory()` | vide l’historique |
| `getInputHistory()` | un `Sample`, historique des entrées **X** |
| `getOutputHistory()` | un `Sample`, historique des sorties **Y** |

**Table 2.** Méthodes de la classe `MemoizeFunction`.


In [10]:
function = ot.PythonFunction(3, 2, simulator)
function = ot.MemoizeFunction(function)

In [11]:
outputVariableOfInterest = ot.CompositeRandomVector(function, inputRandomVector)
sampleSize = 10
outputSample = outputVariableOfInterest.getSample(sampleSize)

Récupère l'historique en entrée.

In [12]:
inputs = function.getInputHistory()
inputs

,v0,v1,v2
0,-0.3426049,-0.689637,1.243087
1,-2.138653,-1.463523,1.102626
2,-0.6212717,0.05683585,-1.010414
3,0.1409458,0.5532786,0.6904337
4,0.3056622,-1.136505,1.692227
5,-1.518799,0.9695327,-1.76564
6,1.860326,1.21488,0.8128723
7,0.3277056,-0.6888172,-2.011791
8,-0.3952833,1.841281,-1.368505
9,0.4561018,-0.1313743,0.5728196


## Fonction symbolique

La classe `SymbolicFunction` peut créer des fonctions symboliques :

- Idée : utiliser une fonction symbolique simple.
- Principe : fournir une chaîne de caractères définissant le calcul.

**Avantages :**

- Amélioration de la performance.
- Calcul automatique du gradient exact, de la Hessienne exacte à condition de ne pas utiliser le mot-clé `var` (voir ci-dessous).

**Inconvénients :**

- Nécessite une formule mathématique « plutôt simple », typiquement une seule ligne (mais cette limitation peut être levée avec ExprTk).

## Fonction symbolique : prototype

**Prototype :**

```python
function = SymbolicFunction(liste_des_entrees, liste_des_formules)
```

où :
- `liste_des_entrees` : une liste de `nbInputs` chaînes de caractères, le nom des variables d’entrées,
- `liste_des_formules` : une liste de `nbOutputs` chaînes de caractères, les formules de calcul.

In [13]:
X1 = ot.Normal(0.0, 1.0)
X2 = ot.Normal(0.0, 1.0)
X3 = ot.Normal(0.0, 1.0)
inputDistribution = ot.JointDistribution((X1, X2, X3))
inputRandomVector = ot.RandomVector(inputDistribution)

In [14]:
function = ot.SymbolicFunction(("X1", "X2", "X3"), ("X1 + X2 + X3", "X1 - X2 * X3"))
function = ot.MemoizeFunction(function)

In [15]:
symbolicRandomVector = ot.CompositeRandomVector(function, inputRandomVector)
sampleSize = 10000
outputSample = symbolicRandomVector.getSample(sampleSize)
empiricalMean = outputSample.computeMean()
print(f"Sample mean = {empiricalMean}")
print(f"True mean   = {trueMean}")

Sample mean = [-0.000337898,-0.00609942]
True mean   = [0,0]


In [16]:
outputs = function.getOutputHistory()
outputs[0:10, :]

,v0,v1
0,-0.08508063,0.8974876
1,0.8350684,0.6082917
2,-1.533106,-0.3603107
3,0.329499,-0.5302618
4,1.697775,0.09262298
5,1.067171,-0.7204318
6,-0.3294656,-0.04315071
7,-1.494857,-0.4512993
8,1.094296,1.906137
9,0.3261065,0.1468117


## Gradient d'une fonction symbolique

Avec la bibliothèque ExprTk, on peut obtenir le gradient exact d'une fonction symbolique sous certaines conditions.

- Si la formule symbolique n'utilise pas le mot-clé `var`, alors le gradient est exact.
- Sinon, le gradient est fondé sur une formule de différences finies.

Cette méthode est à privilégier dès que la formule mathématique est analytique, car elle permet à OpenTURNS d'utiliser son moteur interne (en C++) et de fournir des gradients exacts sans surcoût de calcul par différences finies.


In [17]:
# Sans `var` : le gradient est exact
inputs = ["Q", "Ks", "Zv", "Zm", "Hd", "Zb", "L", "B"]
outputs = ["(Zm - Zv) / L"]
myFunction = ot.SymbolicFunction(inputs, outputs)

# Calcul du gradient : exact
gradient = myFunction.getGradient()
print(gradient)


| d(y0) / d(Q) = 0
| d(y0) / d(Ks) = 0
| d(y0) / d(Zv) = (-1)/(L)
| d(y0) / d(Zm) = (1)/(L)
| d(y0) / d(Hd) = 0
| d(y0) / d(Zb) = 0
| d(y0) / d(L) = (-(((Zm)+(-1*Zv))/(L^2)))
| d(y0) / d(B) = 0



In [18]:
# Avec `var` : le gradient est fondé sur des différences finies
inputs = ["Q", "Ks", "Zv", "Zm", "Hd", "Zb", "L", "B"]
outputs = ["H", "S"]
formula = "var alpha := (Zm - Zv) / L;"
formula += "H := (Q / (Ks * B * sqrt(alpha)))^(3.0 / 5.0);"
formula += "var Zc := H + Zv;"
formula += "var Zd := Zb + Hd;"
formula += "S := Zc - Zd"
myFunction = ot.SymbolicFunction(inputs, outputs, formula)

# Calcul du gradient : différences finies
gradient = myFunction.getGradient()
print(gradient)

No analytical gradient available. Try using finite difference instead.


##  Exercices

### Exercice 1 : une fonction avec 4 entrées

On considère un nouveau modèle, avec une nouvelle variable de sortie
Y3 et une nouvelle variable d’entrée X4 :

$$
\begin{aligned}
Y_1 &= X_1 + X_2 + X_3, \\
Y_2 &= X_1 - X_2 X_3, \\
Y_3 &= 2 X_1 + 3 X_2 + 4 X_4.
\end{aligned}
$$

**Questions**

- Modifier la fonction Python pour simuler le nouveau modèle.
- Ajouter une nouvelle variable $X_4$ de loi normale standard dans le modèle probabiliste.
- Estimer la moyenne de la sortie par Monte-Carlo simple.

### Exercice 2 : gradient d'une fonction Python

La bibliothèque peut calculer la dérivée approchée d’une fonction Python par différences finies. On peut paramétrer la formule de différence utilisée, ainsi que le pas de différentiation de cette formule. De plus, lorsque la matrice Jacobienne est implémentée dans une fonction Python, on peut transmettre cette fonction à OpenTURNS pour qu'il l'utilise.

**Questions**

- Définir la fonction `function` comme dans le sujet, c'est-à-dire avec 3 entrées et 2 sorties.
- Utiliser la méthode `gradient()` de l’objet `function` pour évaluer le gradient $g'(x)$ au point d’entrée $\boldsymbol{x} = (1, 2, 3)^\top$.
- Utiliser la méthode `hessian()` de l’objet `function` pour évaluer la matrice Hessienne de $g$.

- Utiliser les instructions suivantes pour configurer un gradient calculé par une formule de différences finies décentrée, avec un pas $h = 10^{−2}$.

```python
functionImpl = function.getEvaluation()
h = 1.0e-2
gradient = ot.NonCenteredFiniteDifferenceGradient(h, functionImpl)
function.setGradient(gradient)
```

- Évaluer à nouveau le gradient avec la méthode `gradient()` et comparer avec le résultat précédent.
- On peut transmettre à la bibliothèque une fonction Python qui évalue le gradient. Pour cela on peut utiliser la séquence d'appel :
```python
function = ot.PythonFunction(nbInputs, nbOutputs, simulator, gradient=simulatorGradient)
```
où `simulatorGradient` est une fonction Python qui évalue le gradient.

Calculez à la main des dérivées partielles de la fonction $g$ associée à l'exemple. 
Puis définissez la fonction `simulatorGradient` qui évalue la matrice Jacobienne. Puisqu'il y a trois variables d'entrée, la liste renvoyée par `simulatorGradient` doit contenir trois éléments. Chaque élément doit contenir une sous-liste de taille 2 contenant les dérivées de chaque sortie. Enfin, construisez la fonction associée avec l'option `gradient`. 

### Exercice 3 : gestion de l'historique d'une fonction Python

**Questions**

- Observer le changement de la valeur de retour de `isHistoryEnabled()`
- Quelles sont les méthodes qui permettent de récupérer les historiques des entrées et des sorties ?
- Comment avoir le nombre d’appels à la fonction ?
- Utiliser la méthode `clearHistory()` et vérifier que l'historique est vide après cet appel.

### Exercice 4 : fonction symbolique avec 4 entrées

On considère le modèle physique :
$$
\begin{aligned}
Y_1 &= X_1 + X_2 + X_3, \\
Y_2 &= X_1 - X_2 X_3, \\
Y_3 &= 2 X_1 + 3 X_2 + 4 X_4.
\end{aligned}
$$

**Questions**

- Créer une fonction symbolique pour définir ce nouveau modèle.
- Évaluer la sortie du modèle au point $X=(1,2,3,4)^T$.
- Estimer la moyenne de la sortie par Monte-Carlo simple.

### Exercice 5 : fonction symbolique avec paramètres

On considère le modèle physique :
$$
\begin{aligned}
Y_1 &= a X_1 + b X_2, \\
Y_2 &= c X_1 + d X_2,
\end{aligned}
$$

où $a$, $b$, $c$, $d$ sont des paramètres :

- $a = 12$,
- $b = 23$,
- $c = -34$,
- $d = 45$.

**Questions**

- Créer une fonction symbolique pour définir ce modèle en fonction des entrées $a$, $b$, $c$, $d$, $X_1$, $X_2$.
- Utiliser la classe `ParametricFunction` pour définir une fonction prenant en entrée $X_1$ et $X_2$ et dont les paramètres sont $a$, $b$, $c$, $d$. 
- Évaluer la sortie du modèle au point $X=(1,2)^T$.

### Exercice 6 : gradient d'une fonction symbolique

On souhaite vérifier que la bibliothèque peut calculer la dérivée formelle d’une
fonction symbolique. 

**Questions**

- Définir la fonction `functionSymbolic` comme dans l’exemple.
- Créer la variable `gradient` contenant la dérivée exacte de la fonction. Pour cela, utiliser la méthode `getGradient()` de l’objet
`functionSymbolic`. 

- Qu’est-ce qui s’affiche quand on utilise l’instruction suivante ?

```python
print(gradient)
```

- On souhaite évaluer le gradient au point d’entrée $\boldsymbol{x} = (1, 2, 3)$. Utiliser la méthode `gradient()` de l’objet `gradient` pour évaluer $g'(x)$.

### Exercice 7 : gestion des variables intermédiaires dans une fonction symbolique

On peut définir une fonction symbolique dont l'évaluation est fondée sur des valeurs intermédiaires. Ainsi, la sortie n'est pas seulement une fonction explicite des entrées : on peut définir des résultats intermédiaires et les réutiliser dans une ou plusieurs sorties de la fonction. 

Pour cela, il faut utiliser la séquence d'appel suivante :
```python
myFunction = ot.SymbolicFunction(inputs, outputs, formula)
```
où `outputs` est une chaîne de caractères contenant l'expression à évaluer. 

Pour composer cette chaîne de caractères, on peut définir plusieurs expressions, séparées par le caractère ";". De plus, les variables intermédiaires doivent être précédées du mot-clé `var`. 

Par exemple, dans le cas du modèle dont les entrées sont $X_1$ et $X_2$ et les sorties sont $Y_1$ et $Y_2$ :

$$
\begin{aligned}
T   &= X_1 X_2, \\
Y_1 &= X_1 + T, \\
Y_2 &= X_2 - 3T,
\end{aligned}
$$

on peut utiliser l'instruction suivante :

In [19]:
inputs = ["X1", "X2"]
outputs = ["Y1", "Y2"]
formula = "var T := X1 * X2; Y1 := X1 + T; Y2 := X2 - 3 * T"
myFunction = ot.SymbolicFunction(inputs, outputs, formula)
myFunction([1.0, 2.0])

class=Point name=Unnamed dimension=2 values=[3,-4]

Définir le modèle dont les entrées sont $X_1$ et $X_2$ et les sorties sont $Y_1$ et $Y_2$ :
$$
\begin{aligned}
S   &= X_1 + X_2, \\
T   &= X_1 X_2, \\
Y_1 &= S + T, \\
Y_2 &= S T.
\end{aligned}
$$

## Exercice 8 : configurer le nombre de CPUs

L'option `n_cpus` de la classe `PythonFunction` permet de configurer le nombre de processeurs. L'implémentation est fondée sur le module `multiprocessing`. Dans cet exercice, on cherche à observer l'effet de cette option sur la performance du calcul.

Pour observer un changement dans la performance nous nous plaçons dans la situation suivante :

- La fonction possède un grand nombre de variables d'entrée.
- La fonction est coûteuse.

Dans ce but, nous définissons la fonction suivante.

In [20]:
def highDimensionSimulator(x):
    dim = ot.Point(x).getDimension()
    y0 = 0.0
    y1 = 1.0
    for i in range(dim):
        y0 = y0 + math.exp(x[i])
        y1 = y1 * math.exp(x[i])
    y = [y0, y1]
    return y

**Questions**

- Utiliser le module `time` pour mesurer la performance de la fonction sans l'option `n_cpus`. Pour observer une durée de simulation significative, augmentez la taille du plan d'expériences ou le nombre de dimensions.
- Faire de même avec l'option `n_cpus`.
- Quelle différence constatez-vous ?

Une des limites de l'implémentation précédente est qu'elle est fondée sur une boucle `for`, ce qui n'est pas très performant. Une alternative consiste à vectoriser le code de la manière suivante.

```python
def highDimensionSimulatorVectorized(x):
    xArray = np.array(x)
    y0 = np.exp(xArray).sum(axis=1)
    y1 = np.exp(xArray).prod(axis=1)
    return np.column_stack((y0, y1))
```

Dans ce cas, il n'est pas pertinent d'utiliser l'option `n_cpus`, car la vectorisation est déjà prise en charge par NumPy. Comparer la performance des fonctions `highDimensionSimulator` et `highDimensionSimulatorVectorized`. Qu'observez-vous ?

**Remarque.** L'option `n_cpus` doit être utilisée de manière particulière sur Windows. Voir [Multiprocessing runs (Windows)](https://github.com/openturns/openturns/issues/1355). L'implémentation utilise **multiprocessing**. Sur Windows, the point d'entrée du programme doit être protégé par le code `if __name__== ‘__main__’`.